In [1]:
!pip3 install fastapi pandas torch transformers uvicorn google-cloud-storage scikit-learn


In [12]:

from fastapi import FastAPI, UploadFile, HTTPException
import pandas as pd
import torch
import json
import io
from transformers import BartTokenizer

In [11]:


from transformers import BartForConditionalGeneration

_MODEL = None

def get_model():
    global _MODEL

    if _MODEL is None:
        pretrained_bart = BartForConditionalGeneration.from_pretrained('facebook/bart-large')
        checkpoint_path = '/Users/michaelholborn/Documents/SoftwareLocal/monotropism/thoughtx/task1_task2_taskNRv2_finetune_BrainTranslator_skipstep1_b1_20_30_5e-05_5e-07_unique_sent.pt'  # Change to the path of your model
        _MODEL = BrainTranslator(pretrained_bart)
        model_weights = torch.load(checkpoint_path, map_location=torch.device('cpu'))
        _MODEL.load_state_dict(model_weights)
        _MODEL.eval()

    return _MODEL


In [13]:


from transformers import BartTokenizer, BartForConditionalGeneration
import torch.nn as nn
import torch.nn.functional as F

# Define the BrainTranslator model
class BrainTranslator(nn.Module):
    def __init__(self, pretrained_layers, in_feature=840, decoder_embedding_size=1024, additional_encoder_nhead=8, additional_encoder_dim_feedforward=2048):
        super(BrainTranslator, self).__init__()
        
        self.pretrained = pretrained_layers
        self.additional_encoder_layer = nn.TransformerEncoderLayer(d_model=in_feature, nhead=additional_encoder_nhead, dim_feedforward=additional_encoder_dim_feedforward, batch_first=True)
        self.additional_encoder = nn.TransformerEncoder(self.additional_encoder_layer, num_layers=6)
        self.fc1 = nn.Linear(in_feature, decoder_embedding_size)

    def forward(self, input_embeddings_batch, input_masks_batch, input_mask_invert, decoder_input_ids):
        encoded_embedding = self.additional_encoder(input_embeddings_batch, src_key_padding_mask=input_mask_invert)
        encoded_embedding = F.relu(self.fc1(encoded_embedding))
        out = self.pretrained(inputs_embeds=encoded_embedding, attention_mask=input_masks_batch, decoder_input_ids=decoder_input_ids, return_dict=True)
        return out


mask our own masks


In [14]:
import torch
def load_embeddings_from_file(filepath: str) -> torch.Tensor:
    """
    Load embeddings from a given JSON file.

    Parameters:
    - filepath (str): The path to the JSON file containing embeddings.

    Returns:
    - torch.Tensor: A tensor containing the loaded embeddings.
    """
    with open(filepath, 'r') as file:
        embeddings_data = json.load(file)
    return torch.tensor(embeddings_data)

In [15]:
input_embeddings_data=load_embeddings_from_file("./datasets/input_old_model/us-central1_eeg-test_dodadqada_dodadqada_saved_data_input_embeddings_12.json")

In [16]:
def generate_masks_from_embeddings(embeddings: torch.Tensor) -> (torch.Tensor, torch.Tensor):
    """
    Generate attention masks and their inverse for a given embeddings tensor.

    Parameters:
    - embeddings (torch.Tensor): The embeddings tensor.

    Returns:
    - tuple: A tuple containing the attention mask and its inverse.
    """
    # Assuming non-zero embeddings represent valid tokens and zeros represent padding
    attn_mask = (embeddings.sum(dim=-1) != 0).float()
    attn_mask_invert = 1.0 - attn_mask
    return attn_mask, attn_mask_invert

In [17]:
attn_mask,attn_mask_invert=generate_masks_from_embeddings(input_embeddings_data)

In [18]:

import torch

# Convert loaded data to PyTorch tensors
input_embeddings_tensor = torch.tensor(input_embeddings_data)
input_masks_tensor = torch.tensor(attn_mask)
input_mask_invert_tensor = torch.tensor(attn_mask_invert)

input_embeddings_tensor.shape, input_masks_tensor.shape, input_mask_invert_tensor.shape

/var/folders/8m/y9dvzjx95lg__pqq1cypt21h0000gn/T/ipykernel_75901/2687417766.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_embeddings_tensor = torch.tensor(input_embeddings_data)
/var/folders/8m/y9dvzjx95lg__pqq1cypt21h0000gn/T/ipykernel_75901/2687417766.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_masks_tensor = torch.tensor(attn_mask)
/var/folders/8m/y9dvzjx95lg__pqq1cypt21h0000gn/T/ipykernel_75901/2687417766.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_mask_invert_tensor = torch.tensor(attn_mask_inv

(torch.Size([1, 56, 840]), torch.Size([1, 56]), torch.Size([1, 56]))

In [13]:

# Initialize model
pretrained_bart = BartForConditionalGeneration.from_pretrained('facebook/bart-large')
model = BrainTranslator(pretrained_bart)

# Create a placeholder token
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
placeholder_token = tokenizer("<s>", return_tensors="pt")

# Attempt a forward pass
with torch.no_grad():
    try:
        outputs = model(input_embeddings_tensor, input_masks_tensor, input_mask_invert_tensor, placeholder_token["input_ids"])
        result = "Forward pass successful!"
    except Exception as e:
        result = str(e)

In [19]:
result
outputs
# Extract the generated token IDs from the model's outputs
generated_ids = outputs.logits.argmax(dim=-1)
generated_ids

# Decode the token IDs to text
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
generated_text


'is'

In [ ]:

from fastapi import FastAPI, UploadFile, HTTPException
import pandas as pd
import torch
import json
import io
from transformers import BartTokenizer

In [28]:


from transformers import BartForConditionalGeneration

_MODEL = None

def get_model():
    global _MODEL

    if _MODEL is None:
        pretrained_bart = BartForConditionalGeneration.from_pretrained('facebook/bart-large')
        checkpoint_path = '/Users/michaelholborn/Documents/SoftwareLocal/monotropism/thoughtx/task1_task2_taskNRv2_finetune_BrainTranslator_skipstep1_b1_20_30_5e-05_5e-07_unique_sent.pt'  # Change to the path of your model
        _MODEL = BrainTranslator(pretrained_bart)
        model_weights = torch.load(checkpoint_path, map_location=torch.device('cpu'))
        _MODEL.load_state_dict(model_weights)
        _MODEL.eval()

    return _MODEL


def load_embeddings_from_file(filepath: str) -> torch.Tensor:
    """
    Load embeddings from a given JSON file.

    Parameters:
    - filepath (str): The path to the JSON file containing embeddings.

    Returns:
    - torch.Tensor: A tensor containing the loaded embeddings.
    """
    with open(filepath, 'r') as file:
        embeddings_data = json.load(file)
    return torch.tensor(embeddings_data)

def generate_masks_from_embeddings(embeddings: torch.Tensor) -> (torch.Tensor, torch.Tensor):
    """
    Generate attention masks and their inverse for a given embeddings tensor.

    Parameters:
    - embeddings (torch.Tensor): The embeddings tensor.

    Returns:
    - tuple: A tuple containing the attention mask and its inverse.
    """
    # Assuming non-zero embeddings represent valid tokens and zeros represent padding
    attn_mask = (embeddings.sum(dim=-1) != 0).float()
    attn_mask_invert = 1.0 - attn_mask
    return attn_mask, attn_mask_invert


from transformers import BartTokenizer, BartForConditionalGeneration
import torch.nn as nn
import torch.nn.functional as F


""" main architecture for open vocabulary EEG-To-Text decoding"""

# Define the BrainTranslator model
class BrainTranslator(nn.Module):
    def __init__(self, pretrained_layers, in_feature=840, decoder_embedding_size=1024, additional_encoder_nhead=8, additional_encoder_dim_feedforward=2048):
        super(BrainTranslator, self).__init__()
        
        self.pretrained = pretrained_layers
        self.additional_encoder_layer = nn.TransformerEncoderLayer(d_model=in_feature, nhead=additional_encoder_nhead, dim_feedforward=additional_encoder_dim_feedforward, batch_first=True)
        self.additional_encoder = nn.TransformerEncoder(self.additional_encoder_layer, num_layers=6)
        self.fc1 = nn.Linear(in_feature, decoder_embedding_size)

    def forward(self, input_embeddings_batch, input_masks_batch, input_mask_invert, decoder_input_ids):
        encoded_embedding = self.additional_encoder(input_embeddings_batch, src_key_padding_mask=input_mask_invert)
        encoded_embedding = F.relu(self.fc1(encoded_embedding))
        out = self.pretrained(inputs_embeds=encoded_embedding, attention_mask=input_masks_batch, decoder_input_ids=decoder_input_ids, return_dict=True)
        return out


# class BrainTranslator(nn.Module):
#     def __init__(self, pretrained_layers, in_feature = 840, decoder_embedding_size = 1024, additional_encoder_nhead=8, additional_encoder_dim_feedforward = 2048):
#         super(BrainTranslator, self).__init__()
        
#         self.pretrained = pretrained_layers
#         # additional transformer encoder, following BART paper about 
#         self.additional_encoder_layer = nn.TransformerEncoderLayer(d_model=in_feature, nhead=additional_encoder_nhead,  dim_feedforward = additional_encoder_dim_feedforward, batch_first=True)
#         self.additional_encoder = nn.TransformerEncoder(self.additional_encoder_layer, num_layers=6)
        
#         # print('[INFO]adding positional embedding')
#         # self.positional_embedding = PositionalEncoding(in_feature)

#         self.fc1 = nn.Linear(in_feature, decoder_embedding_size)

#     def forward(self, input_embeddings_batch, input_masks_batch, input_masks_invert, target_ids_batch_converted):
#         """input_embeddings_batch: batch_size*Seq_len*840"""
#         """input_mask: 1 is not masked, 0 is masked"""
#         """input_masks_invert: 1 is masked, 0 is not masked"""
        
#         # input_embeddings_batch = self.positional_embedding(input_embeddings_batch) 

#         # use src_key_padding_masks
#         encoded_embedding = self.additional_encoder(input_embeddings_batch, src_key_padding_mask = input_masks_invert) 
        
#         # encoded_embedding = self.additional_encoder(input_embeddings_batch) 
#         encoded_embedding = F.relu(self.fc1(encoded_embedding))
#         out = self.pretrained(inputs_embeds = encoded_embedding, attention_mask = input_masks_batch, return_dict = True, labels = target_ids_batch_converted)                    
        
#         return out



# Initialize model
pretrained_bart = BartForConditionalGeneration.from_pretrained('facebook/bart-large')
model = BrainTranslator(pretrained_bart)


# model = BrainTranslator(pretrained_bart, in_feature = 105*len([
#         "_t1",
#         "_t2",
#         "_a1",
#         "_a2",
#         "_b1",
#         "_b2",
#         "_g1",
#         "_g2"
#     ]), decoder_embedding_size = 1024, additional_encoder_nhead=8, additional_encoder_dim_feedforward = 2048)


# Create a placeholder token
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
placeholder_token = tokenizer("<s>", return_tensors="pt")


In [29]:
running_loss = 0.0

# Iterate over data.
sample_count = 0

target_tokens_list = []
target_string_list = []
pred_tokens_list = []
pred_string_list = []
# Step 2: Create a loop to process each file
results = []
max_file_number=50
for i in range(1, max_file_number + 1):
    # Step 3: Dynamically build the file paths based on the loop index
    input_embeddings_path = f"./datasets/saved_data/input_embeddings_{i}.json"
    print(input_embeddings_path)
    # Step 4: Load the data from these files
    try:
       input_embeddings_data=load_embeddings_from_file(input_embeddings_path)
       input_masks_data,input_mask_invert_data=generate_masks_from_embeddings(input_embeddings_data)
    except FileNotFoundError:
        # If one of the files isn't found, we'll append an error message and continue to the next iteration
        results.append(f"Files for index {i} not found.")
        continue

    # Convert loaded data to PyTorch tensors
    input_embeddings_tensor = torch.tensor(input_embeddings_data)
    input_masks_tensor = torch.tensor(input_masks_data)
    input_mask_invert_tensor = torch.tensor(input_mask_invert_data)
    
    # Step 5: Process the data with the model
    with torch.no_grad():
        try:
            outputs = model(input_embeddings_tensor, input_masks_tensor, input_mask_invert_tensor, placeholder_token["input_ids"])
            # Extract the generated token IDs from the model's outputs
            logits=outputs.logits
            probs = logits[0].softmax(dim = 1)
            values, predictions = probs.topk(1)
            predictions = torch.squeeze(predictions)
            predicted_string = tokenizer.decode(predictions).split('</s></s>')[0].replace('<s>','')
            predictions = predictions.tolist()
            truncated_prediction = []
            for t in predictions:
                if t != tokenizer.eos_token_id:
                    truncated_prediction.append(t)
                else:
                    break
            pred_tokens = tokenizer.convert_ids_to_tokens(truncated_prediction, skip_special_tokens = True)
            # print('predicted tokens:',pred_tokens)
            pred_tokens_list.append(pred_tokens)
            pred_string_list.append(predicted_string)
            print('predicted string:',predicted_string)
            # results.append(generated_text)
        except Exception as e:
            results.append(str(e))

pred_string_list[:10]  # Display the first 10 results for brevity

./datasets/saved_data/input_embeddings_1.json


/var/folders/8m/y9dvzjx95lg__pqq1cypt21h0000gn/T/ipykernel_75901/2822111578.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_embeddings_tensor = torch.tensor(input_embeddings_data)
/var/folders/8m/y9dvzjx95lg__pqq1cypt21h0000gn/T/ipykernel_75901/2822111578.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_masks_tensor = torch.tensor(input_masks_data)
/var/folders/8m/y9dvzjx95lg__pqq1cypt21h0000gn/T/ipykernel_75901/2822111578.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_mask_invert_tensor = torch.tensor(inp

predicted string: It
./datasets/saved_data/input_embeddings_2.json
predicted string:  State
./datasets/saved_data/input_embeddings_3.json
predicted string:  you
./datasets/saved_data/input_embeddings_4.json
predicted string:  Rep
./datasets/saved_data/input_embeddings_5.json
predicted string: is
./datasets/saved_data/input_embeddings_6.json
predicted string: is
./datasets/saved_data/input_embeddings_7.json
predicted string: es
./datasets/saved_data/input_embeddings_8.json
predicted string:  Damage
./datasets/saved_data/input_embeddings_9.json
predicted string: irl
./datasets/saved_data/input_embeddings_10.json
predicted string:  Price
./datasets/saved_data/input_embeddings_11.json
predicted string:  Pier
./datasets/saved_data/input_embeddings_12.json
predicted string: is
./datasets/saved_data/input_embeddings_13.json
predicted string:  Like
./datasets/saved_data/input_embeddings_14.json
predicted string:  State
./datasets/saved_data/input_embeddings_15.json
predicted string: VerVer Bir

['It', ' State', ' you', ' Rep', 'is', 'is', 'es', ' Damage', 'irl', ' Price']

In [30]:
pred_string_list

['It',
 ' State',
 ' you',
 ' Rep',
 'is',
 'is',
 'es',
 ' Damage',
 'irl',
 ' Price',
 ' Pier',
 'is',
 ' Like',
 ' State',
 'VerVer Bird',
 ' Pier',
 'en',
 ' held',
 ' Pier',
 'is',
 ' else',
 'This',
 ' Pepper',
 'ad',
 'ad',
 ' ',
 '</s>',
 ' left',
 ' Plane',
 'is',
 'is',
 'is',
 'is',
 ' must',
 ' Never',
 'I',
 'pe',
 ' I',
 'is',
 'is',
 ' do',
 ' re',
 'ItIt -',
 'II I',
 'ItIt people',
 ' Matt',
 '.',
 '""is',
 'II I',
 'ThisThis him']

In [5]:
checkpoint_path = '/Users/michaelholborn/Documents/SoftwareLocal/monotropism/thoughtx/task1_task2_taskNRv2_finetune_BrainTranslator_skipstep1_b1_20_30_5e-05_5e-07_unique_sent.pt'  # Change to the path of your model

checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
for layer_name, weight in checkpoint.items():
    print(layer_name, weight.shape)

pretrained.final_logits_bias torch.Size([1, 50265])
pretrained.model.shared.weight torch.Size([50265, 1024])
pretrained.model.encoder.embed_tokens.weight torch.Size([50265, 1024])
pretrained.model.encoder.embed_positions.weight torch.Size([1026, 1024])
pretrained.model.encoder.layers.0.self_attn.k_proj.weight torch.Size([1024, 1024])
pretrained.model.encoder.layers.0.self_attn.k_proj.bias torch.Size([1024])
pretrained.model.encoder.layers.0.self_attn.v_proj.weight torch.Size([1024, 1024])
pretrained.model.encoder.layers.0.self_attn.v_proj.bias torch.Size([1024])
pretrained.model.encoder.layers.0.self_attn.q_proj.weight torch.Size([1024, 1024])
pretrained.model.encoder.layers.0.self_attn.q_proj.bias torch.Size([1024])
pretrained.model.encoder.layers.0.self_attn.out_proj.weight torch.Size([1024, 1024])
pretrained.model.encoder.layers.0.self_attn.out_proj.bias torch.Size([1024])
pretrained.model.encoder.layers.0.self_attn_layer_norm.weight torch.Size([1024])
pretrained.model.encoder.layer

In [ ]:
checkpoint1 = torch.load('path_to_checkpoint1.pt', map_location=torch.device('cpu'))
checkpoint2 = torch.load('path_to_checkpoint2.pt', map_location=torch.device('cpu'))

are_weights_same = True
for (layer_name1, weight1), (layer_name2, weight2) in zip(checkpoint1.items(), checkpoint2.items()):
    if not torch.equal(weight1, weight2):
        are_weights_same = False
        print(f"Different weights in layer: {layer_name1}")
        
if are_weights_same:
    print("The weights in both checkpoints are the same.")
else:
    print("The weights in the checkpoints are different.")
